In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [42]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [43]:
from pyomo.environ import *

**1. Formulate this as one of the discrete optimization problems you learnt in IE 684. Clearly describe the formulation with explicitly listing the variables, objective function and constraints**

Let $x_i $ = number of boxes of type $i $ produced. This can be an Non Negative integer. $x_i \in \mathbf{N} \cup \{0\}$

Let $y_i $ is a binary variable indicating whether any boxes of type $i$ are produced. $y_i = 1 $ if $x_i > 0 $, and $y_i = 0 $ otherwise.

This constraint can be enforced using big M method. Meaning,

$$x_i \leq M \cdot y_i \quad \text{for } i = 1, 2, \ldots, 6 $$

Where $M$ is sufficiently large.

Let $v_i$ be volume of box type $i$. Then the objective function, will become:

$$ minimize \quad \sum_{i=1}^{6} \left(1500 \cdot y_i + 100 \cdot v_i \cdot x_i \right)$$

The constraint including the bigM method:

1. The demand should be met.
$$ x_i \geq d_i \quad \forall i = 1,2,... 6 $$
where $d_i$ is demand for box of type $i$

2. $$x_i \leq M \cdot y_i \quad \text{for } i = 1, 2, \ldots, 6 $$


$
\begin{align*}
\ \\
\text{Subject to:} \quad & \\
& 0.15x_1 + 0.25x_2 + 0.35x_3 + 0.5x_4 + 0.6x_5 + 0.75x_6 \geq 300 \\
& 0.25x_2 + 0.35x_3 + 0.5x_4 + 0.6x_5 + 0.75x_6 \geq 450 \\
& 0.35x_3 + 0.5x_4 + 0.6x_5 + 0.75x_6 \geq 80 \\
& 0.5x_4 + 0.6x_5 + 0.75x_6 \geq 500 \\
& 0.6x_5 + 0.75x_6 \geq 150 \\
& 0.75x_6 \geq 800 \\
& x_i \geq 0 \quad \text{for } i = 1,2,3,4,5,6
\end{align*}
$

**2. Solve it using an algorithm you have used previously in IE 684 Lab. List a table of the optimal number of boxes
of each size and state the optimal cost. Don’t use solver.**

In [25]:
# Define the demand, volume, and selling price of each box
boxes = [
    {"demand": 300, "volume": 0.15, "price": 300},
    {"demand": 450, "volume": 0.25, "price": 300},
    {"demand": 80, "volume": 0.35, "price": 350},
    {"demand": 500, "volume": 0.5, "price": 500},
    {"demand": 150, "volume": 0.6, "price": 600},
    {"demand": 800, "volume": 0.75, "price": 800}
]

# Sort the boxes by volume in descending order
sorted_boxes = sorted(boxes, key=lambda x: x["volume"], reverse=True)

# Initialize variables
produced_boxes = [0, 0, 0, 0, 0, 0]  # Number of boxes produced for each size
total_cost = 0

# Greedy algorithm to produce boxes
for box in sorted_boxes:
    index = boxes.index(box)
    demand = box["demand"]
    volume = box["volume"]
    variable_cost = 100 * volume
    fixed_cost = 1500
    available_demand = min(demand, produced_boxes[index - 1])  # Use boxes of larger size to satisfy demand
    produced_boxes[index] = demand - available_demand  # Produce remaining boxes to meet demand
    total_cost += (variable_cost * produced_boxes[index]) + (fixed_cost * produced_boxes[index])

# Output the optimal number of boxes and total cost
print("Optimal Number of Boxes Produced for Each Size:", produced_boxes)
print("Optimal Total Cost:", total_cost)


Optimal Number of Boxes Produced for Each Size: [0, 450, 80, 500, 150, 800]
Optimal Total Cost: 3078050.0


**3. Suppose only 1000 cubic meter of space is available for transport for Flipbakox company. So they may not be able
to fulfill whole demand. Instead they would want to maximize their profit. Formulate this as an optimization
problem. Clearly describe the formulation with explicitly listing the variables, objective function and constraints.**

Let $x_i$ represent the number of boxes of type $i$ produced, where $x_i$ is a non-negative integer $x_i \in \mathbb{N} \cup {0}$.

Let $y_i$ be a binary variable denoting the production status of box type $i$, where $y_i = 1$ if $x_i > 0$, and $y_i = 0$ otherwise.

 The price of product $x_i$ is denoted by $p_i$. The volume of box type $i$ is represented by $v_i$.

To enforce the production constraints using the big M method, the following inequality applies $$x_i \leq M \cdot y_i \quad \text{for } i = 1, 2, \ldots, 6 $$ where $M$ is a sufficiently large constant.

The objective function aims to maximize profit, considering the selling price and the cost of manufacturing:

 $$ maximize \quad \sum_{i=1}^{6} \left(p_i \cdot x_i - (1500 \cdot y_i + 100 \cdot v_i \cdot x_i \right))$$

Under the following constraints:

The total volume of produced boxes must not exceed 1000, i.e.:$$\sum_{j=1}^{6}x_j \cdot v_j \leq 1000 \quad $$
The big M method constraint for each type of box: $$x_i \leq M \cdot y_i \quad \text{for } i = 1, 2, \ldots, 6 $$

**4. Solve the problem. You can use a solver. Write, explain and interpret the results. Your answer should clearly
explain how many boxes of each type is required to be transported.**

In [33]:
boxes = [[1, 300, 0.15, 300],
  [2,450,0.25,300],
  [3 ,80 ,0.35, 350],
  [4 ,500 ,0.5 ,500],
  [5 ,150 ,0.6, 600],
  [6 ,800 ,0.75 ,800]]
demand_in_vol = []
box_no = []
vol_per_box  = []
price = []
demand = []
for ele in boxes:
  demand_in_vol.append(ele[1]*ele[2])
  box_no.append(ele[0])
  vol_per_box.append(ele[2])
  price.append(ele[3])
  demand.append(ele[1])

In [36]:
M = 1000000000

In [37]:
model = ConcreteModel()
model.x = Var(range(6), domain = NonNegativeIntegers)
model.y = Var(range(6), domain = Binary)
model.obj = Objective(expr = sum(( (price[i] - 100*vol_per_box[i])*model.x[i] - 1500*model.y[i] )for i in range(6)), sense = maximize)

model.constraints = ConstraintList()
model.constraints.add(expr = sum(model.x[i]*vol_per_box[i] for i in range(6)) <= 1000)
for i in range(6):
  model.constraints.add(expr = model.x[i] <= M*model.y[i])

result = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)
model.pprint()

2 Var Declarations
    x : Size=6, Index={0, 1, 2, 3, 4, 5}
        Key : Lower : Value  : Upper : Fixed : Stale : Domain
          0 :     0 : 6666.0 :  None : False : False : NonNegativeIntegers
          1 :     0 :    0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :    0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :    0.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :    0.0 :  None : False : False : NonNegativeIntegers
          5 :     0 :    0.0 :  None : False : False : NonNegativeIntegers
    y : Size=6, Index={0, 1, 2, 3, 4, 5}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :

In [41]:
model.obj()


1898310.0

In [46]:
print('Decision Variables for our model : ')
for i in range(6):
    print(f'x{i} : ', model.x[i].value)

Decision Variables for our model : 
x0 :  6666.0
x1 :  0.0
x2 :  0.0
x3 :  0.0
x4 :  0.0
x5 :  0.0


**Here it can be seen that only the box of volume 0.15 is being used to meet all the demands. In this way we are able to minimize the cost required to meet the demands**
